In [ ]:


import pygmmis
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.lines as lines
import matplotlib.cm
import datetime
from functools import partial
import logging
from numpy.random import RandomState
from test_pygmmis import getSelection, plotResults, plotDifferences



# Setup Data 

In [ ]:

# set up test
N = 400             # number of samples
K = 3               # number of components
T = 1               # number of runs
sel_type = "boxWithHole"    # type of selection
disp = 0.5          # additive noise dispersion
bg_amp = 0.0        # fraction of background samples
w = 0.1             # minimum covariance regularization [data units]
cutoff = 5          # cutoff distance between components [sigma]
seed = 1        # seed value
oversampling = 10   # for missing data: imputation samples per observed sample
# show EM iteration results
logging.basicConfig(format='%(message)s',level=logging.INFO)

# define RNG for run
rng = RandomState(seed)





In [ ]:

# draw N points from 3-component GMM
D = 2
gmm = pygmmis.GMM(K=3, D=2)
gmm.amp[:] = np.array([ 0.36060026,  0.27986906,  0.206774])
gmm.amp /= gmm.amp.sum()
print(gmm.amp)
gmm.mean[:,:] = np.array([[ 0.08016886,  0.21300697],
                            [ 0.70306351,  0.6709532 ],
                            [ 0.01087670,  0.852077]])*10
gmm.covar[:,:,:] = np.array([[[ 0.08530014, -0.00314178],
                                [-0.00314178,  0.00541106]],
                                [[ 0.03053402, 0.0125736],
                                [0.0125736,  0.01075791]],
                                [[ 0.00258605,  0.00409287],
                                [ 0.00409287,  0.01065186]]])*100

In [ ]:
# data come from pure GMM model or one with background?
orig = gmm.draw(N, rng=rng)
if bg_amp == 0:
    orig_bg = orig
    bg = None
else:
    footprint = np.array([-10,-10]), np.array([20,20])
    bg = pygmmis.Background(footprint)
    bg.amp = bg_amp
    bg.adjust_amp = True

    bg_size = int(bg_amp/(1-bg_amp) * N)
    orig_bg = np.concatenate((orig, bg.draw(bg_size, rng=rng)))

In [ ]:
# add isotropic errors on data
noisy = orig_bg + rng.normal(0, scale=disp, size=(len(orig_bg), D))

# get observational selection function
omega, ps = getSelection(sel_type, rng=rng)

# apply selection
sel = rng.rand(N) < omega(noisy)
data = noisy[sel]
# single covariance for all samples
covar = disp**2 * np.eye(D)


In [ ]:

# plot data vs true model
plotResults(orig, data, gmm, patch=ps, description="Truth", disp=disp, log=False)


# EM without imputation, deconvolving via extreme convolution 

In [ ]:
#
# repeated runs: store results and logL
l = np.empty(T)
gmms = [pygmmis.GMM(K=K, D=D) for r in range(T)]

# 1) EM without imputation, ignoring errors
start = datetime.datetime.now()
rng = RandomState(seed)
for r in range(T):
    if bg is not None:
        bg.amp = bg_amp
    l[r], _ = pygmmis.fit(gmms[r], data, w=w, cutoff=cutoff, background=bg, rng=rng)
avg = pygmmis.stack(gmms, l)
print ("execution time %ds" % (datetime.datetime.now() - start).seconds)
plotResults(orig, data, avg, patch=ps, description="Standard EM")

# pygmmis with imputation, igoring errors

In [ ]:
start = datetime.datetime.now()
rng = RandomState(seed)
for r in range(T):
    if bg is not None:
        bg.amp = bg_amp
    pygmmis.fit(gmms[r], data, w=w, cutoff=cutoff, background=bg, rng=rng)
    l[r], _ = pygmmis.fit(gmms[r], data, init_method='none', w=w,  cutoff=cutoff, sel_callback=omega,  oversampling=oversampling, background=bg, rng=rng)
avg = pygmmis.stack(gmms, l)
print ("execution time %ds" % (datetime.datetime.now() - start).seconds)
plotResults(orig, data, avg, patch=ps, description="$\mathtt{GMMis}$")


# 4) pygmmis with imputation, incorporating errors


In [ ]:

covar_cb = partial(pygmmis.covar_callback_default, default=np.eye(D)*disp**2)
start = datetime.datetime.now()
rng = RandomState(seed)
for r in range(T):
    if bg is not None:
        bg.amp = bg_amp
    pygmmis.fit(gmms[r], data, w=w, cutoff=cutoff, background=bg, rng=rng)
    l[r], _ = pygmmis.fit(gmms[r], data, covar=covar, init_method='none', w=w, cutoff=cutoff, sel_callback=omega, oversampling=oversampling, covar_callback=covar_cb, background=bg, rng=rng)
avg = pygmmis.stack(gmms, l)
print ("execution time %ds" % (datetime.datetime.now() - start).seconds)
plotResults(orig, data, avg, patch=ps, description="$\mathtt{GMMis}$ & noise deconvolution")

# SNL

### Data for SNL

In [ ]:

batch_size_snl = 32
proposal_sample = batch_size_snl * 10
lr = 0.01
# n_epochs = 100
n_iter = 10000

config = {
    'n_iter': n_iter,
    'lr': lr,
    'proposal_sample': proposal_sample,
    'batch_size': batch_size_snl,
    'n_iter': n_iter,
    'seed': seed,
}


In [ ]:
from Model.Energy import GeneralizedGaussianMixtureEnergy,GeneralizedGaussianMixtureEnergyVectorParam, CircleTruncation, CombineTruncation, MaxMinTruncation
from Model.Proposal import GaussianProposal, UniformProposal, MixtureOfGeneralizedGaussianProposal, MixtureOfGaussianProposal
from Data import get_dataloader_from_data
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader
from Model.Trainer import SNLTrainer
from Model.Energy import MaxMinCensorship
import random

In [ ]:
train_dataset = TensorDataset(torch.tensor(data, dtype=torch.float32))
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = TensorDataset(torch.tensor(orig, dtype=torch.float32))
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=True)


In [ ]:

# proposal_distribution = GaussianProposal(torch.Size([2,]), torch.tensor([0,0], dtype=torch.float32), torch.tensor([10,10], dtype=torch.float32))
# proposal_distribution.mu.data = torch.tensor([5,5], dtype=torch.float32)
# proposal_distribution.sigma.data = torch.tensor([7,7], dtype=torch.float32)
proposal_distribution = UniformProposal(torch.Size([2,]), torch.tensor([0,0], dtype=torch.float32), torch.tensor([10,10], dtype=torch.float32))


### Energy 

In [ ]:
energy = GeneralizedGaussianMixtureEnergy(dim=2, num_cluster=3, learn_pi=True, learn_mu=True, learn_sigma=True)
energy = GeneralizedGaussianMixtureEnergyVectorParam(dim=2, num_cluster=3, learn_pi=True, learn_mu=True, learn_sigma=True)


In [ ]:
# KMeans on Data :
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3, random_state=0).fit(data)
centers = kmeans.cluster_centers_

energy.mu.data = torch.tensor(centers, dtype=torch.float32)

In [ ]:
circle_truncation = CircleTruncation(center = torch.tensor([6.5, 6], dtype=torch.float32), radius=2)
maxmin_truncation = MaxMinTruncation(min = torch.tensor([0,0], dtype=torch.float32), max = torch.tensor([10,10], dtype=torch.float32))
combine_truncation = CombineTruncation([circle_truncation, maxmin_truncation])



In [ ]:
energy.set_truncator(combine_truncation)

# Trainer

## Trainer Uniform

In [ ]:
trainer = SNLTrainer(energy, proposal_distribution, dataloader=train_loader, val_dataloader=train_loader, n_sample_train=proposal_sample, n_sample_test=10000, lr=lr)


In [ ]:
D = 2
gmm = pygmmis.GMM(K=3, D=2)
gmm.amp[:] = torch.nn.functional.log_softmax(energy.logit_pi, dim=-1).exp().detach().numpy()
gmm.amp /= gmm.amp.sum()
gmm.mean[:,:] = energy.mu.detach().numpy()
print(gmm.mean)
gmm.covar[:,:,:] = torch.linalg.inv(energy.get_precision_matrix()).detach().numpy()
# plot data vs true model
plotResults(orig, data, gmm, patch=ps, description="SNL_start", disp=disp, log=True, name="SNL_start", )


In [ ]:
trainer.train(n_iter=n_iter, n_iter_pretrain=1000, plot_every=500)

In [ ]:
D = 2
gmm = pygmmis.GMM(K=3, D=2)
gmm.amp[:] = torch.nn.functional.log_softmax(energy.logit_pi, dim=-1).exp().detach().numpy()
gmm.amp /= gmm.amp.sum()
gmm.mean[:,:] = energy.mu.detach().numpy()
gmm.covar[:,:,:] = torch.linalg.inv(energy.get_precision_matrix()).detach().numpy()
# plot data vs true model
# plotResults(orig[:1], data[:1], gmm, patch=ps, description="SNL", disp=disp)
plotResults(orig, data, gmm, patch=ps, description="SNL", disp=disp, log=True, name="SNL Best", step =trainer.total_step)

### Load best

In [ ]:
trainer.get_best_model()
best_energy = trainer.best_energy
D = 2
gmm = pygmmis.GMM(K=3, D=2)
gmm.amp[:] = torch.nn.functional.log_softmax(best_energy.logit_pi, dim=-1).exp().detach().numpy()
gmm.amp /= gmm.amp.sum()
gmm.mean[:,:] = best_energy.mu.detach().numpy()
gmm.covar[:,:,:] = torch.linalg.inv(best_energy.get_precision_matrix()).detach().numpy()
# plot data vs true model
# plotResults(orig[:1], data[:1], gmm, patch=ps, description="SNL", disp=disp)
plotResults(orig, data, gmm, patch=ps, disp = disp, log=True, name="SNL Uniform", step=trainer.best_step)

## Trainer Mixture

In [ ]:
logit = torch.nn.functional.log_softmax(energy.logit_pi, dim=-1).exp().detach()
mean = energy.mu.detach()
covar = torch.linalg.inv(energy.get_precision_matrix()).detach()
proposal_v2 = MixtureOfGeneralizedGaussianProposal(input_size = torch.Size((2,)), logit_pi = logit, mu = mean, sigma = covar)
trainer.proposal = proposal_v2

In [ ]:
trainer.train(n_iter=n_iter, n_iter_pretrain=0, plot_every=500)

In [ ]:
D = 2
gmm = pygmmis.GMM(K=3, D=2)
gmm.amp[:] = torch.nn.functional.log_softmax(energy.logit_pi, dim=-1).exp().detach().numpy()
gmm.amp /= gmm.amp.sum()
gmm.mean[:,:] = energy.mu.detach().numpy()
gmm.covar[:,:,:] = torch.linalg.inv(energy.get_precision_matrix()).detach().numpy()
# plot data vs true model
# plotResults(orig[:1], data[:1], gmm, patch=ps, description="SNL", disp=disp)
plotResults(orig, data, gmm, patch=ps, description="SNL Self Proposal", disp=disp, log=True, name="SNL_self_proposal", step =trainer.total_step)

### Load best

In [ ]:
trainer.get_best_model()
best_energy = trainer.best_energy
D = 2
gmm = pygmmis.GMM(K=3, D=2)
gmm.amp[:] = torch.nn.functional.log_softmax(best_energy.logit_pi, dim=-1).exp().detach().numpy()
gmm.amp /= gmm.amp.sum()
gmm.mean[:,:] = best_energy.mu.detach().numpy()
gmm.covar[:,:,:] = torch.linalg.inv(best_energy.get_precision_matrix()).detach().numpy()
# plot data vs true model
# plotResults(orig[:1], data[:1], gmm, patch=ps, description="SNL", disp=disp)
plotResults(orig, data, gmm, patch=ps, log=True, description="SNL_self_proposal_best", disp=disp, step = trainer.best_step)